# GPT4 Assistant Actionable Item Analysis
#### Background
The City Auditor's office identified two problematic themes in their report regarding item actionability: "Specifically, some recommendations contained language that makes it difficult for the recommendation to be implemented as they are not specific, measurable, and actionable. In addition, many recommendations are not currently tracked."

#### Question
Can we use the OpenAI assistant API to audit each recommendation looking for language that is not specific, measureable, and actionable?

#### Approach
- Create an OpenAI assistant that can analyze the dataset.
- Instruct the assistant to determine whether each entry contains specific, measurable, and actionable language.
- Log the assistant's observations in an updated dataset for further analysis.

## Create OpenAI Assistant and Upload Dataset

In [4]:
%reload_ext autoreload
%autoreload 2

import sys
import os
file_dir = os.path.abspath('')
sys.path.append(file_dir)

from openai import OpenAI
import assistant_helper
import pandas as pd

ai_instructions = "You are a data analyst and statistician. You are cleaning and improving the attached CSV dataset."
assistant = assistant_helper.create_openai_assistant('Displacement_and_Gentrification_Recommendation_Inventory.csv', 'AI Analyst', ai_instructions)

client = OpenAI()


---

## Verify "Actionable?" Data Accuracy with Assistant
- Have assistant analyze each row and determine actionibility
- Log confidence in analysis from 0-1 in separate column
- Log reasoning in separate column

Hopefully this will reveal outlying actionability decisions for further analysis.

To prevent the model from being influenced by the existing "Actionable?" values, I removed the original column from the data.

In [5]:

# Create a new thread
thread = client.beta.threads.create()

# Add message to thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
Please conduct an active analysis on each entry within the 'Recommendation/Resolution' column of the dataset. Your role includes reading, interpreting, and evaluating the text to determine if each entry is specific, measurable, and actionable. You are expected to fill in the analysis directly, without setting up placeholders for a manual review later.

Here's how to proceed:

1. Analysis Execution:
   - For each recommendation or resolution, analyze the language used. Do not insert placeholder text; provide actual analysis based on the text's content.

2. Determination and Justification:
   - Create a 'Determination' column with the options 'Meets Criteria' or 'Does Not Meet Criteria'.
   - In the 'Justification' column, write a detailed explanation for your determination. Cite specific language from the recommendation that either meets or fails to meet the criteria for specificity, measurability, and actionability.

3. Direct Engagement:
   - Engage with the content of each entry as you would with any text analysis task, applying your language processing capabilities to assess and determine the adequacy of the recommendations based on the criteria mentioned.

Your analysis is to be entered directly into the dataset, with each 'Determination' and 'Justification' reflecting your immediate assessment of the text. Please proceed with the substantive evaluation and update the dataset with your findings.

Provide the updated dataset in a CSV file in your first message for further analysis.

"""
)

# start a run on the thread
messages = assistant_helper.start_run_on_thread(client, thread.id, assistant.id)
print(messages.data[0].content[0].text.value)



Completed!          
The analysis of each entry within the 'Recommendation/Resolution' column of the dataset has been completed based on the text's specificity, measurability, and actionability. The 'Determination' and 'Justification' columns have been added to the dataset, reflecting the immediate assessment of the text.

The updated dataset has been saved as a CSV file, which you can download from the link below:

[Updated Dataset CSV](sandbox:/mnt/data/updated_dataset.csv)


### Process CSV Output

In [6]:
import io

# Access returned CSV
first_message = messages.data[0]
if first_message.file_ids:
    file_id = first_message.file_ids[0]
    print(f"File identified: {file_id}")

else:
    print("No file identified in first message.")

# Read binary stream
file_content = client.files.content(file_id)
file_bytes = file_content.read()
file_bytes_io = io.BytesIO(file_bytes)

# Read as CSV
implementation_category_df = pd.read_csv(file_bytes_io)

# Save CSV to out folder
file_name = os.path.join("out", "AI_Actionability.csv") 
implementation_category_df.to_csv(file_name, encoding='utf-8', index=False)
print(f"Output CSV saved in {file_name}")

display(implementation_category_df)

File identified: file-yOZJNFGJP9YJs3dbDr1qDO5r
Output CSV saved in out/AI_Actionability.csv


,Year (CY),Type,Source,Summary,Recommendation/Resolution,Implementing Department,Action Category,Direct/Indirect?,Targeted at Homelessness?,Actionable?,Status of the Recommendation/Resolution,Determination,Justification
0,2011,OCA Audit,Affordable Housing: Rental Housing and Acquisi...,NaN,In order to support the efficiency and effecti...,NHCD,Monitoring,Indirect,No,NaN,NaN,Does Not Meet Criteria,The recommendation is specific. It contains me...
1,2011,OCA Audit,Affordable Housing: Rental Housing and Acquisi...,NaN,The NHCD Director should improve existing proc...,NHCD,Recommendations for process improvements,Indirect,No,NaN,NaN,Does Not Meet Criteria,The recommendation is specific. It lacks measu...
2,2012,OCA Audit,Austin Energy Customer Assistance Programs Audit,NaN,AE management should work with Council to iden...,Austin Energy,Financing,Indirect,No,NaN,NaN,Does Not Meet Criteria,The recommendation is specific. It lacks measu...
3,2012,OCA Audit,Austin Energy Customer Assistance Programs Audit,NaN,AE management should work with the Plus 1 Prog...,Austin Energy,Financing,Indirect,No,NaN,NaN,Meets Criteria,The recommendation is specific. It contains me...
4,2015,OCA Audit,Austin Energy Low-Income Weatherization Progra...,NaN,5. Austin Energy management should continue to...,Austin Energy,Financing,Indirect,No,NaN,NaN,Meets Criteria,The recommendation is specific. It contains me...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,2013,Council Resolution,20130808-052,Resolution initiating amendments to the City C...,• City Council initiates amendments to City Co...,City Manager,Preserving and expanding the supply of afforda...,Direct,No,Actionable,Implemented,Does Not Meet Criteria,The recommendation lacks specificity. It conta...
537,2002,Task Force,Gentrification Implications of Historic Zoning...,NaN,6. Educate residents about their legal rights ...,Council,Recommendations for planning and approach,Direct,No,Actionable,Status unknown: commission and taskforce recom...,Does Not Meet Criteria,The recommendation lacks specificity. It lacks...
538,2004,Council Resolution,20041202-59,A resolution adopting the City's Affordable Ho...,The following strategies for reporting Catellu...,City Manager,Controlling land for community development,Direct,No,Actionable,"Status unknown: according to CMO, item predate...",Does Not Meet Criteria,The recommendation lacks specificity. It conta...
539,2016,Council Resolution,20160303-018,Directing the City Manager to initiate amendme...,• City Manager directed to initiate amendments...,City Manager,Financing,Direct,No,Actionable,Implemented,Does Not Meet Criteria,The recommendation lacks specificity. It lacks...
